# 笔记本 6.2: Baichuan-13B

## 6.2.1 概述

本笔记本展示了如何使用[BigDL-LLM](https://github.com/intel-analytics/BigDL/tree/main/python/llm) API 在低成本 PC 上（无需独立显卡）运行[Baichuan-13B](https://github.com/baichuan-inc/Baichuan-13B) 中文推理。Baichuan-13B 是百川智能科技继 [Baichuan-7B](https://github.com/baichuan-inc/baichuan-7B) 之后开发的一款开源、商业化的大规模语言模型。Baichuan-13B 还可在以下[链接](https://huggingface.co/baichuan-inc/Baichuan-13B-Chat)中的[Huggingface models](https://huggingface.co/models)中找到。

## 6.2.2 安装

首先，在准备好的环境中安装 BigDL-LLM。有关环境配置的最佳实践，请参阅本教程的 [第二章](../ch_2_Environment_Setup/)。

In [ ]:
!pip install bigdl-llm[all]

# Baichuan-13B-Chat 进行生成所需的额外软件包
!pip install -U transformers_stream_generator

all 选项用于安装 BigDL-LLM 所需的其他软件包。

## 6.2.3 加载模型与 Tokenizer

### 6.2.3.1 加载模型

使用 BigDL-LLM API加载低精度优化（INT4）的 Baichuan 模型以降低资源成本，这会将模型中的相关层转换为 INT4 格式，


> **注意**
>
> 您可以指定参数 `model_path` 为 Huggingface repo id 或本地模型路径。

In [ ]:
from bigdl.llm.transformers import AutoModelForCausalLM

model_path = "baichuan-inc/Baichuan-13B-Chat"
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             load_in_4bit=True,
                                             trust_remote_code=True)

### 6.2.3.2 加载 Tokenizer

LLM 推理也需要一个 tokenizer。它用于将输入文本编码为张量，然后输入 LLM，并将 LLM 输出的张量解码为文本。您可以使用 [Huggingface transformers](https://huggingface.co/docs/transformers/index) API 直接加载 tokenizer。它可以与 BigDL-LLM 加载的模型无缝配合使用。

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path,
                                          trust_remote_code=True)

## 6.2.4 推理

### 6.2.4.1 创建 Prompt 模板

在生成之前，您需要创建一个 prompt 模板。这里我们给出了一个用于提问和回答的 prompt 模板示例。您也可以根据自己的模型调整 prompt。

In [7]:
BAICHUAN_PROMPT_FORMAT = "<human>{prompt} <bot>"

### 6.2.4.2 生成

接下来，您可以使用加载的模型与 tokenizer 生成输出。

> **注意**
>
> `generate` 函数中的 `max_new_tokens` 参数定义了预测的最大 token 数量。

In [10]:
import time
import torch

prompt = "AI是什么？"
n_predict = 128
with torch.inference_mode():
        prompt = BAICHUAN_PROMPT_FORMAT.format(prompt=prompt)
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        st = time.time()
        # 如果您选择的模型能够利用之前的 key/value attentions 来提高解码速度，
        # 但其模型配置中的 `"use_cache": false`，
        # 则必须在 `generate` 函数中明确设置 `use_cache=True`，
        # 以便利用 BigDL-LLM INT4 优化获得最佳性能。
        output = model.generate(input_ids,
                                max_new_tokens=n_predict)
        end = time.time()
        output_str = tokenizer.decode(output[0], skip_special_tokens=True)
        print(f'Inference time: {end-st} s')
        print('-'*20, 'Output', '-'*20)
        print(output_str)

Inference time: xxxx s
-------------------- Output --------------------
<human>AI是什么？ <bot>人工智能(Artificial Intelligence，简称AI)是指由人制造出来的系统所表现出来的智能，通常是通过计算机系统实现的。这种智能来源于计算机程序和数据处理能力，可以模拟、扩展和辅助人类的认知功能。
